In [ ]:
!uv pip install -q "torch>=2.2" torchvision --index-url https://download.pytorch.org/whl/cu121/
!uv pip install -q --upgrade diffusers transformers accelerate safetensors xformers
!uv pip install -q "pillow<12.0" opencv-python matplotlib

print("All packages installed. Restart the runtime before using")

All packages installed. Restart the runtime before using


In [ ]:
import os
import numpy as np
import time
import gc
import torch
from PIL import Image
from IPython.display import HTML, display

# prefer float 16 whenever possible
from diffusers import (
    StableDiffusionPipeline,
    StableDiffusionXLPipeline,
    DDIMScheduler
)

/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [ ]:
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
print(f"Model cache directory: {cache_dir}")

Device: cpu
Model cache directory: /content/huggingface_cache


In [ ]:
generator = None
if device == 'cuda':
  generator = torch.Generator(device='cuda').manual_seed(42)
else:
  generator = torch.Generator(device='cpu').manual_seed(42)

In [ ]:
# prompts
prompt_sd1_5 = ""
prompt_sdxl = ""

In [ ]:
def free_cuda_cache():
  gc.collect()
  if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("CUDA cache cleared.")
  else:
    print("CUDA is not available.")

In [ ]:
# helper to safely load a stable diffusion pipeline
def safe_from_pretrained(cls_name, repo_id, **kwargs):
  # tries to load a pipeline and if the safety checker init causes typeError or similar, retries with safety_checker=True
  try:
    pipe = cls_name.from_pretrained(repo_id, **kwargs)
    return pipe
  except Exception as e:
    print(f"Error loading {repo_id}: {e}")
    print(f"Retrying with safety_checker=None and requires_safety_checker=False....")
    kwargs["safety_checker"] = None
    kwargs["requires_safety_checker"] = False
    pipe = cls_name.from_pretrained(repo_id, **kwargs)
    return pipe
  except Exception as e:
    print(f"Error loading {repo_id}: {e}")
    raise e

In [ ]:
# cache/ download models(one-time)
print("Step 1: Caching/ Downloading models to loacl cache_dir")
free_cuda_cache()

t0 = time.time()
print("Dowloading (caching) runwayml/stavle-diffusion-v1-5...")
try:
  #use dtype arg(diffusers accepts dtype)
  StableDiffusionPipeline.from_pretrained(
      "runwayml/stable-diffusion-v1-5",
      cache_dir=cache_dir,
      torch_dtype=torch.float16,
      safety_checker=None,
      requires_safety_checker=False
  )
  print("Cached SD 1.5")
except Exception as e:
  print(f"Error loading runwayml/stable-diffusion-v1-5: {e}")

Step 1: Caching/ Downloading models to loacl cache_dir
CUDA is not available.
Dowloading (caching) runwayml/stavle-diffusion-v1-5...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Cached SD 1.5


In [ ]:
print("Dowloading (caching) stabilityai/stable-diffusion-xl-base-1.0...")
try:
  StableDiffusionXLPipeline.from_pretrained(
      "stabilityai/stable-diffusion-xl-base-1.0",
      torch_dtype=torch.float16,
      cache_dir=cache_dir,
      variant="fp_16",
      use_safetensors=True,
      safety_checker=None,
      requires_safety_checker=False
  )
  print("Cached SDXL base")
except Exception as e:
  print(f"Error loading stabilityai/stable-diffusion-xl-base-1.0: {e}")



Dowloading (caching) stabilityai/stable-diffusion-xl-base-1.0...


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Error loading stabilityai/stable-diffusion-xl-base-1.0: You are trying to load model files of the `variant=fp_16`, but no such modeling files are available. 


In [ ]:
# 2. Load SD1.5 and generate(local cached)
print("Step 2: Loading SD1.5 from local cache_dir")
free_cuda_cache()

try:
  pipe_sd1_5 = StableDiffusionPipeline.from_pretrained(
      StableDiffusionPipeline,
      "runwayml/stable-diffusion-v1-5",
      cache_dir=cache_dir,
      torch_dtype=torch.float16,
  )
except Exception as e:
  print(f"Error loading runwayml/stable-diffusion-v1-5: {e}")
  raise e

In [ ]:
# display the results
print("\n==== Results ======")
if image_ddpm:
  print()